In [1]:
import pandas as pd
import pickle

Columns to keep
- National Provider Identifier -- 1000 non-null int64
- Last Name/Organization Name of the Provider -- 1000 non-null object
- Entity Type of the Provider -- 1000 non-null object
- City of the Provider -- 1000 non-null object
- Zip Code of the Provider -- 1000 non-null int64
- State Code of the Provider -- 1000 non-null object
- Provider Type -- 1000 non-null object
- Place of Service -- 1000 non-null object
- HCPCS Code -- 1000 non-null object
- HCPCS Description -- 1000 non-null object
- Number of Services -- 1000 non-null float64
- Number of Medicare Beneficiaries -- 1000 non-null int64
- Number of Distinct Medicare Beneficiary/Per Day Services -- 1000 non-null int64
- Average Medicare Allowed Amount -- 1000 non-null float64
- ADD:  Year (in each df on import)

In [4]:
%%time
#option 1
cols = ['National Provider Identifier',
'Last Name/Organization Name of the Provider',
'Entity Type of the Provider', 
'City of the Provider',
'Zip Code of the Provider', 
'State Code of the Provider',
'Provider Type',
'Place of Service',
'HCPCS Code',
'HCPCS Description',
'Number of Services',
'Number of Medicare Beneficiaries',
'Number of Distinct Medicare Beneficiary/Per Day Services',
'Average Medicare Allowed Amount']

payment_rows = []
for chunk in pd.read_csv('../data/Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2017.csv',
                         usecols = cols,
                         chunksize = 1000):
        chunk['year'] = 2017
        payment_rows.append(chunk)
        
df_payments_2017 = pd.concat(payment_rows, ignore_index=True)

CPU times: user 1min 19s, sys: 18.9 s, total: 1min 38s
Wall time: 1min 39s


In [5]:
df_payments_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9847443 entries, 0 to 9847442
Data columns (total 15 columns):
 #   Column                                                    Dtype  
---  ------                                                    -----  
 0   National Provider Identifier                              int64  
 1   Last Name/Organization Name of the Provider               object 
 2   Entity Type of the Provider                               object 
 3   City of the Provider                                      object 
 4   Zip Code of the Provider                                  object 
 5   State Code of the Provider                                object 
 6   Provider Type                                             object 
 7   Place of Service                                          object 
 8   HCPCS Code                                                object 
 9   HCPCS Description                                         object 
 10  Number of Services            

In [8]:
%%time
#option 2

cols = ['National Provider Identifier',
        'Last Name/Organization Name of the Provider',
        'Entity Type of the Provider',
        'City of the Provider',
        'Zip Code of the Provider',
        'State Code of the Provider',
        'Provider Type',
        'Place of Service',
        'Number of Services',
        'Number of Medicare Beneficiaries',
        'Number of Distinct Medicare Beneficiary/Per Day Services',
        'Average Medicare Allowed Amount']

payment_rows =[]
for chunk in pd.read_csv('../data/Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2017.csv', 
                         chunksize = 1000, usecols = cols):
    payment_rows.append(chunk)
    
df_payments_2017 = pd.concat(payment_rows, ignore_index=True)
df_payments_2017['year'] = 2017

CPU times: user 1min 5s, sys: 18.3 s, total: 1min 23s
Wall time: 1min 23s


In [10]:
%%time
#option 3 without chunks and without for loop
cols = ['National Provider Identifier',
        'Last Name/Organization Name of the Provider',
        'Entity Type of the Provider',
        'City of the Provider',
        'Zip Code of the Provider',
        'State Code of the Provider',
        'Provider Type',
        'Place of Service',
        'Number of Services',
        'Number of Medicare Beneficiaries',
        'Number of Distinct Medicare Beneficiary/Per Day Services',
        'Average Medicare Allowed Amount']

df_payments_2017 = pd.read_csv('../data/Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2017.csv', 
                               usecols = cols, low_memory = False)
df_payments_2017['year'] = 2017

CPU times: user 27.2 s, sys: 8.76 s, total: 35.9 s
Wall time: 36.7 s


In [11]:
print(df_payments_2017.shape)
df_payments_2017.head()

(9847443, 13)


,National Provider Identifier,Last Name/Organization Name of the Provider,Entity Type of the Provider,City of the Provider,Zip Code of the Provider,State Code of the Provider,Provider Type,Place of Service,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,year
0,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,100.0,96,100,73.398800,2017
1,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,26.0,25,26,100.080000,2017
2,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,52.0,51,52,136.380000,2017
3,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,59.0,59,59,190.363729,2017
4,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,16.0,16,16,101.680000,2017


In [18]:
#make all columns lowercase & replace spaces with _ and / with _
df_payments_2017.columns = map(str.lower,df_payments_2017.columns)
df_payments_2017.columns = df_payments_2017.columns.str.replace(" ", "_")
df_payments_2017.columns = df_payments_2017.columns.str.replace("/", "_")


df_payments_2017.head()

,national_provider_identifier,last_name_organization_name_of_the_provider,entity_type_of_the_provider,city_of_the_provider,zip_code_of_the_provider,state_code_of_the_provider,provider_type,place_of_service,number_of_services,number_of_medicare_beneficiaries,number_of_distinct_medicare_beneficiary_per_day_services,average_medicare_allowed_amount,year
0,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,100.0,96,100,73.398800,2017
1,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,26.0,25,26,100.080000,2017
2,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,52.0,51,52,136.380000,2017
3,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,59.0,59,59,190.363729,2017
4,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,16.0,16,16,101.680000,2017


In [19]:
df_payments_2017.to_pickle('../data/df_payments_2017.pkl')

#### Option 3 is the fastest to read in data

In [20]:
%%time
#read in year 2016
cols = ['National Provider Identifier',
        'Last Name/Organization Name of the Provider',
        'Entity Type of the Provider',
        'City of the Provider',
        'Zip Code of the Provider',
        'State Code of the Provider',
        'Provider Type',
        'Place of Service',
        'Number of Services',
        'Number of Medicare Beneficiaries',
        'Number of Distinct Medicare Beneficiary/Per Day Services',
        'Average Medicare Allowed Amount']

df_payments_2016 = pd.read_csv('../data/Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2016.csv', 
                               usecols = cols)
df_payments_2016['year'] = 2016

<string>:2: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 26.3 s, sys: 2.74 s, total: 29.1 s
Wall time: 29.3 s


In [22]:
print(df_payments_2016.shape)
df_payments_2016.head()

(9714896, 13)


,national_provider_identifier,last_name_organization_name_of_the_provider,entity_type_of_the_provider,city_of_the_provider,zip_code_of_the_provider,state_code_of_the_provider,provider_type,place_of_service,number_of_services,number_of_medicare_beneficiaries,number_of_distinct_medicare_beneficiary_per_day_services,average_medicare_allowed_amount,year
0,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,57.0,55,57,72.743158,2016
1,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,38.0,38,38,135.010000,2016
2,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,23.0,23,23,189.239565,2016
3,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,20.0,20,20,100.750000,2016
4,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,96.0,87,96,136.250000,2016


In [21]:
df_payments_2016.columns = map(str.lower,df_payments_2016.columns)
df_payments_2016.columns = df_payments_2016.columns.str.replace(" ", "_")
df_payments_2016.columns = df_payments_2016.columns.str.replace("/", "_")


df_payments_2016.head()

,national_provider_identifier,last_name_organization_name_of_the_provider,entity_type_of_the_provider,city_of_the_provider,zip_code_of_the_provider,state_code_of_the_provider,provider_type,place_of_service,number_of_services,number_of_medicare_beneficiaries,number_of_distinct_medicare_beneficiary_per_day_services,average_medicare_allowed_amount,year
0,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,57.0,55,57,72.743158,2016
1,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,38.0,38,38,135.010000,2016
2,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,23.0,23,23,189.239565,2016
3,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,20.0,20,20,100.750000,2016
4,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,96.0,87,96,136.250000,2016


In [23]:
df_payments_2016.to_pickle('../data/df_payments_2016.pkl')

In [33]:
%%time
#read in year 2015
cols = ['National Provider Identifier',
        'Last Name/Organization Name of the Provider',
        'Entity Type of the Provider',
        'City of the Provider',
        'Zip Code of the Provider',
        'State Code of the Provider',
        'Provider Type',
        'Place of Service',
        'Number of Services',
        'Number of Medicare Beneficiaries',
        'Number of Distinct Medicare Beneficiary/Per Day Services',
        'Average Medicare Allowed Amount']

df_payments_2015 = pd.read_csv('../data/Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2015.csv', 
                               usecols = cols)
df_payments_2015['year'] = 2015

<string>:2: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 27.7 s, sys: 4.13 s, total: 31.9 s
Wall time: 33 s


In [34]:
print(df_payments_2015.shape)
df_payments_2015.head()

(9497892, 13)


,National Provider Identifier,Last Name/Organization Name of the Provider,Entity Type of the Provider,City of the Provider,Zip Code of the Provider,State Code of the Provider,Provider Type,Place of Service,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,year
0,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,23.0,23.0,23.0,72.680000,2015
1,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,18.0,18.0,18.0,135.850000,2015
2,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,59.0,58.0,59.0,101.365085,2015
3,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,132.0,130.0,132.0,139.010455,2015
4,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,220.0,215.0,220.0,205.185955,2015


In [35]:
df_payments_2015.columns = map(str.lower,df_payments_2015.columns)
df_payments_2015.columns = df_payments_2015.columns.str.replace(" ", "_")
df_payments_2015.columns = df_payments_2015.columns.str.replace("/", "_")


df_payments_2015.head()

,national_provider_identifier,last_name_organization_name_of_the_provider,entity_type_of_the_provider,city_of_the_provider,zip_code_of_the_provider,state_code_of_the_provider,provider_type,place_of_service,number_of_services,number_of_medicare_beneficiaries,number_of_distinct_medicare_beneficiary_per_day_services,average_medicare_allowed_amount,year
0,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,23.0,23.0,23.0,72.680000,2015
1,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,18.0,18.0,18.0,135.850000,2015
2,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,59.0,58.0,59.0,101.365085,2015
3,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,132.0,130.0,132.0,139.010455,2015
4,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,220.0,215.0,220.0,205.185955,2015


Drop blank row in 2015 file - idenified this in data.cms.gov file

In [36]:
df_payments_2015[df_payments_2015.national_provider_identifier == 1]

,national_provider_identifier,last_name_organization_name_of_the_provider,entity_type_of_the_provider,city_of_the_provider,zip_code_of_the_provider,state_code_of_the_provider,provider_type,place_of_service,number_of_services,number_of_medicare_beneficiaries,number_of_distinct_medicare_beneficiary_per_day_services,average_medicare_allowed_amount,year
7205022,1,CPT copyright 2014 American Medical Associatio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015


In [37]:
df_payments_2015 = df_payments_2015.drop(labels = 7505022)

In [40]:
df_payments_2015.to_pickle('../data/df_payments_2015.pkl')